In [12]:
import pandas as pd
import torch
from torchvision import models, transforms
from PIL import Image
import numpy as np
import os

In [ ]:
model = models.segmentation.deeplabv3_resnet101(pretrained=True).eval()

input_dir = "images" 
output_dir = "removed_background"

os.makedirs(output_dir, exist_ok=True)

transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406],
                         std=[0.229, 0.224, 0.225])
])

for filename in os.listdir(input_dir):
    if filename.lower().endswith((".jpg", ".jpeg", ".png")):
        try:
            image_path = os.path.join(input_dir, filename)
            image = Image.open(image_path).convert("RGB")
            resized = image.resize((520, 520))

            input_tensor = transform(resized).unsqueeze(0)

            with torch.no_grad():
                output = model(input_tensor)['out'][0]
            mask = output.argmax(0).byte().cpu().numpy()

            person_mask = (mask == 15).astype(np.uint8)
            np_image = np.array(resized)
            np_image[person_mask == 0] = [0, 0, 0]

            result = Image.fromarray(np_image)
            result.save(os.path.join(output_dir, filename))
            print(f"Saved: {filename}")
        except Exception as e:
            print(f"Failed on {filename}: {e}")


C:\Users\USER\AppData\Roaming\Python\Python311\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
C:\Users\USER\AppData\Roaming\Python\Python311\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=DeepLabV3_ResNet101_Weights.COCO_WITH_VOC_LABELS_V1`. You can also use `weights=DeepLabV3_ResNet101_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


Saved: 000000001268_sitting.jpg
Saved: 000000005503_standing.jpg
Saved: 000000011149_standing.jpg
Saved: 000000011197_standing.jpg
Saved: 000000013177_sitting.jpg
Saved: 000000013659_sitting.jpg
Saved: 000000014226_sitting.jpg
Saved: 000000016249_sitting.jpg
Saved: 000000017436_sitting.jpg
Saved: 000000017905_standing.jpg
Saved: 000000018491_walking_running.jpg
Saved: 000000018837_sitting.jpg
Saved: 000000021167_standing.jpg
Saved: 000000022371_sitting.jpg
Saved: 000000023126_walking_running.jpg
Saved: 000000029397_standing.jpg
Saved: 000000032901_standing.jpg
Saved: 000000039480_walking_running.jpg
Saved: 000000047819_standing.jpg
Saved: 000000049259_sitting.jpg
Saved: 000000050145_walking_running.jpg
Saved: 000000050811_sitting.jpg
Saved: 000000052507_walking_running.jpg
Saved: 000000054654_standing.jpg
Saved: 000000054931_walking_running.jpg
Saved: 000000057597_walking_running.jpg
Saved: 000000060347_sitting.jpg
Saved: 000000060932_standing.jpg
Saved: 000000061584_sitting.jpg
Saved:

In [ ]:
import torch
from torch import nn, optim
from torchvision import datasets, models, transforms
from torch.utils.data import DataLoader
import os

# Data directories
train_dir = "data/train"
val_dir = "data/val"

# Preprocessing
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406],
                         std=[0.229, 0.224, 0.225])
])

train_data = datasets.ImageFolder(train_dir, transform=transform)
val_data = datasets.ImageFolder(val_dir, transform=transform)

train_loader = DataLoader(train_data, batch_size=32, shuffle=True)
val_loader = DataLoader(val_data, batch_size=32)

# Choose model: ResNet50 or MobileNetV2
def get_model(model_type="resnet"):
    if model_type == "resnet":
        model = models.resnet50(pretrained=True)
        in_features = model.fc.in_features
        model.fc = nn.Linear(in_features, 2)
    elif model_type == "mobilenet":
        model = models.mobilenet_v2(pretrained=True)
        in_features = model.classifier[1].in_features
        model.classifier[1] = nn.Linear(in_features, 2)
    return model

model = get_model("mobilenet")  # or "resnet"

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

# Loss and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=1e-4)

# Training loop
epochs = 5
for epoch in range(epochs):
    model.train()
    running_loss = 0
    for inputs, labels in train_loader:
        inputs, labels = inputs.to(device), labels.to(device)
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
    print(f"Epoch {epoch+1}, Loss: {running_loss/len(train_loader):.4f}")

# Save model
torch.save(model.state_dict(), "sitting_standing_model.pth")
